# 【第1回_Beginner限定コンペ】銀行の顧客ターゲティング

顧客の属性情報などから定期預金キャンペーンの反応率を予測しよう。

https://signate.jp/competitions/292

Deep Table編

## ライブラリインポート

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import lightgbm as lgb

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, classification_report

from deeptables.models.deeptable import DeepTable, ModelConfig
from deeptables.models.deepnets import DeepFM, xDeepFM, DCN, PNN, WideDeep, AutoInt, AFM, FGCNN

import tensorflow as tf

import optuna

# Optuna 並列処理 マルチGPU
# https://github.com/optuna/optuna/issues/1365
#from multiprocessing import Manager
#from joblib import parallel_backend

#from dask.distributed import Client, wait
#from dask_cuda import LocalCUDACluster


In [2]:
# マルチGPUチェック
#cluster = LocalCUDACluster()
#client = Client(cluster)

#n_gpus = 2

## 関数

In [3]:
def reduce_mem_usage(df):
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

In [4]:
def df_stats(df):
    stats = []
    for col in df.columns:
        stats.append((col,
                      df[col].nunique(),
                      df[col].value_counts().index[0],
                      df[col].value_counts().values[0],
                      df[col].isnull().sum() * 100 / df.shape[0],
                      df[col].value_counts(normalize=True, dropna=False).values[0] * 100,
                      df[col].dtype))
    return pd.DataFrame(stats, columns=['カラム名', 'カラムごとのユニーク値数', '最も出現頻度の高い値', '最も出現頻度の高い値の出現回数', '欠損損値の割合', '最も多いカテゴリの割合', 'Type'])

## データ読み込み・前処理

In [5]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
submit_df = pd.read_csv('submit_sample.csv', header=None)

In [6]:
# Pseudo Labeling
#test_p1_df = pd.read_csv('test_p1.csv')

#train_df = pd.concat([train_df, test_p1_df])

In [7]:
df_list = [train_df, test_df]

for df in df_list:
    df['job'] = df['job'].map({'unknown': 1, 'technician': 2, 'blue-collar': 3, 'services': 4, 'entrepreneur': 5, 'admin.': 6, 'management': 7, 'housemaid': 8, 'self-employed': 9, 'unemployed': 10, 'retired': 11, 'student': 12})
    
    df['marital'] = df['marital'].map({'married': 2, 'divorced':1, 'single': 0})
    #df.drop(['marital'], axis=1, inplace=True)

    df['education'] = df['education'].map({'tertiary': 3, 'secondary': 2, 'primary': 1, 'unknown': 0})
    #df.drop(['education'], axis=1, inplace=True)

    #df['default'] = df['default'].map({'yes': 1, 'no': 0})
    df.drop(['default'], axis=1, inplace=True)

    df['housing'] = df['housing'].map({'yes': 1, 'no': 0})
    #df.drop(['housing'], axis=1, inplace=True)
    
    df['loan'] = df['loan'].map({'yes': 1, 'no': 0})
    #df.drop(['loan'], axis=1, inplace=True)

    df['contact'] = df['contact'].map({'telephone': 2, 'cellular': 1, 'unknown': 0})
    #df.drop(['contact'], axis=1, inplace=True)

    df['poutcome'] = df['poutcome'].map({'success': 3, 'unknown': 2, 'failure': 1, 'other': 0})
    #df['p_label_mean'] = np.log(df['poutcome'].map(p_label_mean))
    #df.drop(['poutcome'], axis=1, inplace=True)
    
    df['month'] = df['month'].map({'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'may': 5, 'jun': 6, 'jul': 7, 'aug': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12})
    #df.drop(['day', 'month'], axis=1, inplace=True)
    
    # Feb 30 とかあって、正確には変換できない
    # → データの Feb 30 を Mar 1 に変換した(他にも、 2/31, 6/31, 11/31)
    #df['dayofyear'] = df['month'] * 31 + df['day']
    df['datetime'] = pd.to_datetime('2012/' + df['month'].astype(str).str.pad(2,fillchar='0') + '/' + df['day'].astype(str).str.pad(2,fillchar='0'), format='%Y/%m/%d')
    df['dayofyear'] = df['datetime'].dt.dayofyear
    #df['dayofweek'] = df['datetime'].dt.dayofweek
    df.drop(['datetime'], axis=1, inplace=True)

    df['duration'] = np.log(df['duration'] + 1)

    #df['bpp'] = np.log((df['balance'] - df['balance'].min()) / (df['pdays'] + 2) + 1)
    #df['cdp'] = (df['campaign'] - df['previous']) * df['duration']
    
    df.drop(['pdays'], axis=1, inplace=True)
    df.drop(['balance'], axis=1, inplace=True)
    
    df.drop(['id'], axis=1, inplace=True)

In [8]:
y = train_df.pop('y')

In [9]:
train_df = reduce_mem_usage(train_df)
test_df = reduce_mem_usage(test_df)

Memory usage of dataframe is 2.89 MB
Memory usage after optimization is: 0.41 MB
Decreased by 85.7%
Memory usage of dataframe is 1.93 MB
Memory usage after optimization is: 0.28 MB
Decreased by 85.7%


In [10]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27100 entries, 0 to 27099
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   age        27100 non-null  int8   
 1   job        27100 non-null  int8   
 2   marital    27100 non-null  int8   
 3   education  27100 non-null  int8   
 4   housing    27100 non-null  int8   
 5   loan       27100 non-null  int8   
 6   contact    27100 non-null  int8   
 7   day        27100 non-null  int8   
 8   month      27100 non-null  int8   
 9   duration   27100 non-null  float16
 10  campaign   27100 non-null  int8   
 11  previous   27100 non-null  int8   
 12  poutcome   27100 non-null  int8   
 13  dayofyear  27100 non-null  int16  
dtypes: float16(1), int16(1), int8(12)
memory usage: 423.6 KB


In [11]:
train_df.describe()

,age,job,marital,education,housing,loan,contact,day,month,duration,campaign,previous,poutcome,dayofyear
count,27100.000000,27100.000000,27100.000000,27100.000000,27100.000000,27100.000000,27100.000000,27100.000000,27100.000000,2.710000e+04,27100.000000,27100.000000,27100.000000,27100.000000
mean,36.073284,5.152509,1.386162,2.046125,0.583727,0.127269,0.788007,16.700443,6.003542,inf,1.775830,0.085720,1.855683,168.623579
std,7.816417,2.669990,0.872384,0.727044,0.492949,0.333281,0.498535,8.576252,2.135158,7.719727e-01,0.950045,0.365889,0.467181,65.155774
min,22.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000e+00,1.000000,0.000000,0.000000,3.000000
25%,31.000000,3.000000,0.000000,2.000000,0.000000,0.000000,0.000000,8.000000,5.000000,4.804688e+00,1.000000,0.000000,2.000000,136.000000
50%,33.000000,5.000000,2.000000,2.000000,1.000000,0.000000,1.000000,17.000000,5.000000,5.070312e+00,1.000000,0.000000,2.000000,148.000000
75%,37.000000,7.000000,2.000000,2.000000,1.000000,0.000000,1.000000,26.000000,7.000000,5.847656e+00,2.000000,0.000000,2.000000,199.000000
max,90.000000,12.000000,2.000000,3.000000,1.000000,1.000000,2.000000,31.000000,12.000000,8.031250e+00,5.000000,3.000000,3.000000,336.000000


In [12]:
df_stats(train_df)

,カラム名,カラムごとのユニーク値数,最も出現頻度の高い値,最も出現頻度の高い値の出現回数,欠損損値の割合,最も多いカテゴリの割合,Type
0,age,42,31.000000,4464,0.0,16.472325,int8
1,job,11,3.000000,5957,0.0,21.981550,int8
2,marital,3,2.000000,17565,0.0,64.815498,int8
3,education,4,2.000000,15955,0.0,58.874539,int8
4,housing,2,1.000000,15819,0.0,58.372694,int8
5,loan,2,0.000000,23651,0.0,87.273063,int8
6,contact,3,1.000000,19147,0.0,70.653137,int8
7,day,30,27.000000,4129,0.0,15.236162,int8
8,month,12,5.000000,11232,0.0,41.446494,int8
9,duration,140,5.070312,5759,0.0,21.250923,float16


## Deep Table

In [13]:
class Objective():
    def __init__(self):
        self.best_pred = None
        self._pred = None

    def __call__(self, trial):
        #nets = trial.suggest_categorical('nets', [DeepFM, xDeepFM, DCN, PNN, WideDeep, AutoInt, AFM, FGCNN])
        output_dim = trial.suggest_int('output_dim', 4, 20)
        output_use_bias = bool(trial.suggest_int('output_use_bias', 0, 1))
        use_residual = bool(trial.suggest_int('use_residual', 0, 1))
        use_bias = bool(trial.suggest_int('use_bias', 0, 1))
        direct = bool(trial.suggest_int('direct', 0, 1))
        reduce_D = bool(trial.suggest_int('reduce_D', 0, 1))
        
        conf = ModelConfig(
            nets=xDeepFM, 
            metrics=['AUC'], 
            auto_discrete=True,
            auto_categorize=True,
            cat_exponent=0.5,
            monitor_metric='val_auc',
            earlystopping_patience=3,
            stacking_op='concat',
            output_use_bias=output_use_bias,
            fixed_embedding_dim=True,
            embeddings_regularizer='l2',
            embeddings_output_dim=output_dim,
            embedding_dropout=0.,
            dnn_params={
               'hidden_units':(
                   (2**8, 0., True),
                   (2**7, 0., True),
                   (2**6, 0., True),
                   (2**5, 0., True),
                   (2**4, 0., True),
                   (2**3, 0., True),
                   (1, 0., True),
                   ), #hidden_units
               'dnn_activation':'relu'
            },
            cin_params={
                'cross_layer_size': (128, 128),
                'activation': 'relu',
                'use_residual': use_residual,
                'use_bias': use_bias,
                'direct': direct,
                'reduce_D': reduce_D,
            }
            
        )
        
        dt = DeepTable(config = conf)

        oof_proba, eval_proba, test_proba = dt.fit_cross_validation(
            train_df,
            y,
            X_eval=None,
            X_test=test_df,
            num_folds=5,
            stratified=False,
            random_state=22,
            iterators=None, 
            batch_size=24,
            epochs=100,
            verbose=0,
            callbacks=[],
            n_jobs=1,
        )
        
        self._pred = test_proba
        
        AUC = roc_auc_score(y, oof_proba)
        return AUC
    
    def callback(self, study, trial):
        if study.best_trial == trial:
            self.best_pred = self._pred

In [14]:
objective = Objective()
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20, callbacks=[objective.callback])

Start cross validation
2 class detected, {0, 1}, so inferred as a [binary classification] task
Preparing features cost:0.10061502456665039
Imputation cost:0.1183621883392334
Categorical encoding cost:0.19483518600463867


/home/ubuntu/work/deeptable/.venv/lib/python3.6/site-packages/sklearn/preprocessing/_discretization.py:202: UserWarning:

Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.

/home/ubuntu/work/deeptable/.venv/lib/python3.6/site-packages/sklearn/preprocessing/_discretization.py:202: UserWarning:

Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.

/home/ubuntu/work/deeptable/.venv/lib/python3.6/site-packages/sklearn/preprocessing/_discretization.py:202: UserWarning:

Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.

/home/ubuntu/work/deeptable/.venv/lib/python3.6/site-packages/sklearn/preprocessing/_discretization.py:202: UserWarning:

Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.

/home/ubuntu/work/deeptable/.venv/lib/python3.6/site-pac

Discretization cost:0.14536261558532715
fit_transform cost:0.5716981887817383
transform X_test
transform_X cost:3.994781017303467
Iterators:KFold(n_splits=5, random_state=22, shuffle=True)
Injected a callback [EarlyStopping]. monitor:val_auc, patience:3, mode:max

Fold:1

2 Physical GPUs, 2 Logical GPUs
>>>>>>>>>>>>>>>>>>>>>> Model Desc <<<<<<<<<<<<<<<<<<<<<<< 
---------------------------------------------------------
inputs:
---------------------------------------------------------
['all_categorical_vars: (27)', 'input_continuous_all: (14)']
---------------------------------------------------------
embeddings:
---------------------------------------------------------
input_dims: [44, 13, 5, 6, 4, 4, 5, 32, 14, 142, 7, 6, 6, 6, 5, 4, 3, 3, 3, 3, 3, 4, 4, 5, 3, 3, 3]
output_dims: [18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18]
dropout: 0.0
---------------------------------------------------------
dense: dropout: 0
batch_normal

/home/ubuntu/work/deeptable/.venv/lib/python3.6/site-packages/tensorflow/python/framework/indexed_slices.py:432: UserWarning:

Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.



Restoring model weights from the end of the best epoch.
Epoch 00010: early stopping
Fold 1 fitting over.
Fold 1 scoring over.
Save model to:dt_output/dt_20200831 054049_dnn_nets_linear_cin_nets/dnn_nets_linear_cin_nets-kfold-1.h5.

Fold:2

2 Physical GPUs, 2 Logical GPUs
>>>>>>>>>>>>>>>>>>>>>> Model Desc <<<<<<<<<<<<<<<<<<<<<<< 
---------------------------------------------------------
inputs:
---------------------------------------------------------
['all_categorical_vars: (27)', 'input_continuous_all: (14)']
---------------------------------------------------------
embeddings:
---------------------------------------------------------
input_dims: [44, 13, 5, 6, 4, 4, 5, 32, 14, 142, 7, 6, 6, 6, 5, 4, 3, 3, 3, 3, 3, 4, 4, 5, 3, 3, 3]
output_dims: [18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18]
dropout: 0.0
---------------------------------------------------------
dense: dropout: 0
batch_normalization: False
------------------

...

/home/ubuntu/work/deeptable/.venv/lib/python3.6/site-packages/sklearn/preprocessing/_discretization.py:202: UserWarning:

Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.

/home/ubuntu/work/deeptable/.venv/lib/python3.6/site-packages/sklearn/preprocessing/_discretization.py:202: UserWarning:

Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.

/home/ubuntu/work/deeptable/.venv/lib/python3.6/site-packages/sklearn/preprocessing/_discretization.py:202: UserWarning:

Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.

/home/ubuntu/work/deeptable/.venv/lib/python3.6/site-packages/sklearn/preprocessing/_discretization.py:202: UserWarning:

Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.

/home/ubuntu/work/deeptable/.venv/lib/python3.6/site-pac

Discretization cost:0.1463000774383545
fit_transform cost:0.5612804889678955
transform X_test
transform_X cost:3.9181437492370605
Iterators:KFold(n_splits=5, random_state=22, shuffle=True)
Injected a callback [EarlyStopping]. monitor:val_auc, patience:3, mode:max

Fold:1

2 Physical GPUs, 2 Logical GPUs
>>>>>>>>>>>>>>>>>>>>>> Model Desc <<<<<<<<<<<<<<<<<<<<<<< 
---------------------------------------------------------
inputs:
---------------------------------------------------------
['all_categorical_vars: (27)', 'input_continuous_all: (14)']
---------------------------------------------------------
embeddings:
---------------------------------------------------------
input_dims: [44, 13, 5, 6, 4, 4, 5, 32, 14, 142, 7, 6, 6, 6, 5, 4, 3, 3, 3, 3, 3, 4, 4, 5, 3, 3, 3]
output_dims: [17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17]
dropout: 0.0
---------------------------------------------------------
dense: dropout: 0
batch_normal

/home/ubuntu/work/deeptable/.venv/lib/python3.6/site-packages/tensorflow/python/framework/indexed_slices.py:432: UserWarning:

Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.



Restoring model weights from the end of the best epoch.
Epoch 00012: early stopping
Fold 1 fitting over.
Fold 1 scoring over.
Save model to:dt_output/dt_20200831 102113_dnn_nets_linear_cin_nets/dnn_nets_linear_cin_nets-kfold-1.h5.

Fold:2

2 Physical GPUs, 2 Logical GPUs
>>>>>>>>>>>>>>>>>>>>>> Model Desc <<<<<<<<<<<<<<<<<<<<<<< 
---------------------------------------------------------
inputs:
---------------------------------------------------------
['all_categorical_vars: (27)', 'input_continuous_all: (14)']
---------------------------------------------------------
embeddings:
---------------------------------------------------------
input_dims: [44, 13, 5, 6, 4, 4, 5, 32, 14, 142, 7, 6, 6, 6, 5, 4, 3, 3, 3, 3, 3, 4, 4, 5, 3, 3, 3]
output_dims: [17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17]
dropout: 0.0
---------------------------------------------------------
dense: dropout: 0
batch_normalization: False
------------------

[I 2020-08-31 10:38:52,195] Trial 19 finished with value: 0.8539539035555249 and parameters: {'output_dim': 17, 'output_use_bias': 0, 'use_residual': 1, 'use_bias': 1, 'direct': 1, 'reduce_D': 0}. Best is trial 3 with value: 0.8607667760070288.


In [15]:
print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Number of finished trials: 20
Best trial:
  Value: 0.8607667760070288
  Params: 
    output_dim: 20
    output_use_bias: 1
    use_residual: 0
    use_bias: 0
    direct: 0
    reduce_D: 0


## 評価

In [16]:
optuna.importance.get_param_importances(study)

OrderedDict([('output_dim', 0.46350065674029634),
             ('use_bias', 0.2437481673429274),
             ('reduce_D', 0.13739192454403018),
             ('output_use_bias', 0.06021326283155797),
             ('direct', 0.055952495915994306),
             ('use_residual', 0.03919349262519398)])

## 推論

In [17]:
objective.best_pred

array([0.79993695, 0.13332988, 0.02634035, ..., 0.05121368, 0.00163651,
       0.100538  ], dtype=float32)

In [18]:
submit_df[1] = objective.best_pred
submit_df.to_csv('submit-dt.csv', header=False, index=False)